In [ ]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-
'''
@File    :   convert_dicom.ipynb
@Time    :   2025/05/08 10:49:07
@Author  :   Neutrin 
'''

# here put the import lib

import numpy as np
import matplotlib.pyplot as plt
import pydicom 
import png
from imageio import imread
from wand.image import Image
import pandas as pd
import time
import cv2
import os
import tqdm

In [ ]:
def save_dicom_image_as_png(dicom_filename, png_filename, bitdepth=16):
    """
    将DICOM格式的16位乳房X光片保存为重新缩放的16位PNG文件。
    :param dicom_filename: 输入DICOM文件的路径。
    :param png_filename: 输出PNG文件的路径。
    :param bitdepth: 输入图像的位深度。对于12位乳房X光片，设置为12。
                     对于16位乳房X光片，设置为16，等等。
                     请确保使用正确的位深度！
    """
    # 使用 dcmread 而不是 read_file
    image = pydicom.dcmread(dicom_filename).pixel_array
    # 将乳房X光片调整为您期望的分辨率。
    image = cv2.resize(image, dsize=(896,1152), interpolation = cv2.INTER_CUBIC)
    with open(png_filename, 'wb') as f:
        writer = png.Writer(
            height=image.shape[0],
            width=image.shape[1],
            bitdepth=bitdepth,
            greyscale=True
        )
        writer.write(f, image.tolist())
        
        print("完成！")

In [42]:
def save_dicom_mask_as_png(dicom_filename, png_filename, bitdepth=8):
    """
    将DICOM格式的掩码文件保存为可见的PNG文件。
    
    :param dicom_filename: 输入DICOM掩码文件的路径。
    :param png_filename: 输出PNG文件的路径。
    :param bitdepth: 输出PNG的位深度，对于掩码通常8位就足够了。
    """
    # 读取DICOM文件
    mask = pydicom.dcmread(dicom_filename).pixel_array
    
    # 检查掩码是否为二值图像
    unique_values = np.unique(mask)
    print(f"掩码中的唯一值: {unique_values}")
    
    # 将掩码调整为标准分辨率
    mask = cv2.resize(mask, dsize=(896, 1152), interpolation=cv2.INTER_NEAREST)
    
    # 对掩码进行处理，确保非零值可见
    # 如果是二值掩码，将非零值转为255（确保可见）
    if len(unique_values) <= 2:
        mask = (mask > 0).astype(np.uint8) * 255
    else:
        # 如果有多个值，可以尝试进行归一化或线性拉伸
        if mask.max() > 0:
            mask = (mask.astype(float) / mask.max() * 255).astype(np.uint8)
    
    # # 用matplotlib查看处理后的掩码并保存（调试用）
    # plt.figure(figsize=(6, 8))
    # plt.imshow(mask, cmap='gray')
    # plt.title(f"掩码预览: {os.path.basename(dicom_filename)}")
    # plt.colorbar()
    # plt.savefig(png_filename.replace(".png", "_preview.jpg"))
    # plt.close()
    
    # 保存为PNG
    cv2.imwrite(png_filename, mask)
    print(f"掩码已保存为: {png_filename}")

In [46]:
def convert_roi_masks_to_png(csv_file, output_dir):
    """
    将CSV文件中img_path列的DICOM文件转换为PNG文件，并更新CSV中的路径
    
    :param csv_file: 包含img_path列的CSV文件路径
    :param output_dir: 存储转换后PNG文件的目录
    :return: 包含更新后路径的DataFrame
    """
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 读取CSV文件
    df = pd.read_csv(csv_file)
    
    # 检查是否存在img_path列
    if 'roi_path' not in df.columns:
        raise ValueError(f"CSV文件 {csv_file} 中不存在 'roi_path' 列")
    
    # 为每个DICOM文件创建一个新路径并进行转换
    new_paths = []
    
    for idx, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="转换文件"):
        dicom_path = row['roi_path']
        
        # 从路径中提取文件名，不带扩展名
        filename = os.path.basename(dicom_path).split('.')[0]
        # 使用图像ID构建唯一的文件名
        image_id = row['img']
        png_filename = f"{image_id}_{filename}.png"
        png_path = os.path.join(output_dir, png_filename)
        
        try:
            # 转换DICOM为PNG
            save_dicom_mask_as_png(dicom_path, png_path)
            # 更新路径
            new_paths.append(png_path)
        except Exception as e:
            print(f"处理 {dicom_path} 时出错: {str(e)}")
            # 如果转换失败，保留原始路径
            new_paths.append(dicom_path)
    
    # 更新DataFrame中的路径
    df['img_path'] = new_paths
    
    # 保存更新后的CSV
    output_csv = os.path.join(output_dir, os.path.basename(csv_file).replace('.csv', '_png.csv'))
    df.to_csv(output_csv, index=False)
    
    print(f"转换完成！更新后的CSV文件已保存为：{output_csv}")
    return df

# 示例用法
if __name__ == "__main__":
    # 设置CSV文件路径和输出目录
    csv_file = "E:/Dataset/CBIS-DDSM/mask_csv/calc-training.csv"
    output_dir = "E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training"
    
    # 执行转换
    updated_df = convert_roi_masks_to_png(csv_file, output_dir)

转换文件:   0%|          | 2/1546 [00:00<03:31,  7.31it/s]

掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_00005_RIGHT_CC_1_1-1.png
掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_00005_RIGHT_MLO_1_1-2.png


转换文件:   0%|          | 4/1546 [00:00<03:41,  6.96it/s]

掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_00007_LEFT_CC_1_1-2.png
掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_00007_LEFT_MLO_1_1-2.png


转换文件:   0%|          | 6/1546 [00:00<03:26,  7.48it/s]

...

掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_02572_LEFT_MLO_1_1-2.png


转换文件: 100%|█████████▉| 1545/1546 [04:01<00:00,  4.44it/s]

掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_02584_LEFT_CC_1_1-2.png


转换文件: 100%|██████████| 1546/1546 [04:01<00:00,  6.39it/s]

掩码中的唯一值: [  0 255]
掩码已保存为: E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\Calc-Training_P_02584_LEFT_MLO_1_1-2.png
转换完成！更新后的CSV文件已保存为：E:/Dataset/CBIS-DDSM/New_CBDDSM/roi/calc-training\calc-training_png.csv
